In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn.functional as F
import numpy as np

from ssgd import StreamingSGD

# Model definition

A StreamingSGD compatible model now needs to be able to "detach" layers as well as gather input/output and gradients. It also needs a list of layers. See below for implementation example. In the future we want to implement this using PyTorch hooks and modules.

In [3]:
class ExampleNet(torch.nn.Module):
    def __init__(self):
        super(ExampleNet, self).__init__()
        
        self.input_layer = torch.nn.Conv2d(3, 3, kernel_size=3)
        self.layers = [self.input_layer]

        for i in range(5):  # use 9 for 8194 x 8194 images
            self.add_block(i)
        
        final_conv5 = torch.nn.Conv2d(3, 1, kernel_size=10)
        self.add_module("final", final_conv5)
        
        self.layers.extend([final_conv5])
        
    def add_block(self, i):
        conv1 = torch.nn.Conv2d(3, 3, kernel_size=3)
        conv2 = torch.nn.Conv2d(3, 3, kernel_size=3)
        conv3 = torch.nn.Conv2d(3, 3, kernel_size=3)
        maxpool = torch.nn.MaxPool2d(2, stride=2)
        
        self.add_module("conv1-" + str(i), conv1)
        self.add_module("conv2-" + str(i), conv2)
        self.add_module("conv3-" + str(i), conv3)
        self.add_module("maxpool-" + str(i), maxpool)

        self.layers.extend([conv1, conv2, conv3, maxpool])
        
    def forward(self, x, stop_at_layer=None, start_at_layer=None):
        stop_index = 14 if stop_at_layer else -1
        start_index = 14 if start_at_layer else 0
        
        for i, layer in enumerate(self.layers[start_index:]):
            if i == stop_index:
                break
            if i + 1 == len(self.layers[start_index:]):
                x = layer(x)
                x = x.view(-1, 1)
                x = F.sigmoid(x)
            else:
                x = F.relu(layer(x))
        return x

model = ExampleNet()
# model = model.double()

Weight initialization; we use positive values to generate large gradients, better for testing if final gradients are correct.

In [4]:
for i, layer in enumerate(model.modules()):
    if isinstance(layer, torch.nn.Conv2d):
        layer.weight.data.fill_(0.04)
        layer.bias.data.zero_()

In [5]:
print(model)

ExampleNet(
  (input_layer): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv1-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-0): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv1-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv1-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv1-3): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-3): Conv2d(3, 3, kerne

# Configurations

In [6]:
stop_index = 14  # use 21 for 8194x8194 images
img_size = 514  # try 8194, see last segment for details

cuda = False  # execute this notebook on the GPU
verbose = True  # enable / disable logging
divide_in = 4  # tip: use 25 for 8194x8194 when memory constraint

# Configure streaming SGD

In [7]:
if cuda:
    model.cuda()

In [8]:
sCNN = StreamingSGD(model, stream_to_layer='conv1-3', 
               input_shape=(1, 3, img_size, img_size), 
               divide_in=divide_in, 
               cuda=cuda, 
               verbose=verbose)

Feature map to be reconstructed shape: (56.0, 56.0)
Feature map divided in tile sizes: (14.0, 14.0)
Tile size forward: (178.0, 178.0)
Tile size backward (for forward pass): (242.0, 242.0)
*** Approximate memory reduction of streaming: 77.8% ***


# Generate random image and fake label

In the current implementation the whole image needs to be able to fit in memory (RAM).

In [9]:
image = torch.FloatTensor(3, img_size, img_size).normal_(0, 1)
target = torch.FloatTensor(1, 1).fill_(0)

# image = image.double()
# target = target.double()

if cuda:
    target = target.cuda()

In [10]:
image_var = torch.autograd.Variable(image)
# image_var = image_var.double()

In [11]:
criterion = torch.nn.BCELoss()

In [12]:
output, feature_map = sCNN.forward(image_var)

100%|██████████| 16/16 [00:00<00:00, 148.72it/s]

Doing forward pass...


In [13]:
output

Variable containing:
 0.9907
[torch.FloatTensor of size 1x1]

In [14]:
loss = criterion(output, torch.autograd.Variable(target)); loss

Variable containing:
 4.6831
[torch.FloatTensor of size 1]

In [15]:
full_gradients = sCNN.backward(image_var, feature_map, loss)

  6%|▋         | 1/16 [00:00<00:01,  8.09it/s]

Doing backward pass...


100%|██████████| 16/16 [00:01<00:00, 14.08it/s]

Everything filled:
 True


"Everything filled" means that all gradients were reconstructed succesfully!

Save the gradients of the conv2d layer to compare with normal SGD:

In [16]:
streaming_conv_gradients = []

for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            streaming_conv_gradients.append(layer.weight.grad.clone()) 

# Compare to normal SGD

Reset the gradients and perform a normal for backward pass.

In [17]:
for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            layer.weight.grad.data.zero_()
            layer.bias.grad.data.zero_()

In [18]:
output = model.forward(image_var[None]);

In [19]:
output

Variable containing:
 0.9907
[torch.FloatTensor of size 1x1]

This output should be the same as the streaming SGD output, if so the loss will also be the same:

In [20]:
loss = criterion(output, torch.autograd.Variable(target)); loss

Variable containing:
 4.6831
[torch.FloatTensor of size 1]

Here we do a normal PyTorch backward pass:

In [21]:
loss.backward()

# Compare the gradients of the conv2d layers

In [22]:
normal_conv_gradients = []

for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            normal_conv_gradients.append(layer.weight.grad) 

In [23]:
for i in range(len(streaming_conv_gradients)):
    print("Layer", i, "\taverage gradient size:", torch.mean(streaming_conv_gradients[i].data))

Layer 0 	average gradient size: 1.42917592437179
Layer 1 	average gradient size: 1.4291759022959956
Layer 2 	average gradient size: 1.4291759332021077
Layer 3 	average gradient size: 1.4291759287869488
Layer 4 	average gradient size: 1.4291759376172666
Layer 5 	average gradient size: 1.4291758802202013
Layer 6 	average gradient size: 1.4291759287869488
Layer 7 	average gradient size: 1.4291758978808369
Layer 8 	average gradient size: 1.4291759332021077
Layer 9 	average gradient size: 1.4291759155414723
Layer 10 	average gradient size: 1.4291758802202013
Layer 11 	average gradient size: 1.4291759067111545
Layer 12 	average gradient size: 1.4291759287869488
Layer 13 	average gradient size: 1.4291758404837713
Layer 14 	average gradient size: 1.4291758139928181
Layer 15 	average gradient size: 1.4291759464475844
Layer 16 	average gradient size: 0.3858775034546852


In [25]:
for i in range(len(streaming_conv_gradients)):
    max_diff = torch.max(torch.abs(streaming_conv_gradients[i].data - 
                         normal_conv_gradients[i].data))
    print("Layer", i, "\tmax difference between gradients:", max_diff)

Layer 0 	max difference between gradients: 1.6689300537109375e-06
Layer 1 	max difference between gradients: 1.3113021850585938e-06
Layer 2 	max difference between gradients: 2.1457672119140625e-06
Layer 3 	max difference between gradients: 1.6689300537109375e-06
Layer 4 	max difference between gradients: 8.344650268554688e-07
Layer 5 	max difference between gradients: 5.960464477539062e-07
Layer 6 	max difference between gradients: 5.960464477539062e-07
Layer 7 	max difference between gradients: 4.76837158203125e-07
Layer 8 	max difference between gradients: 3.5762786865234375e-07
Layer 9 	max difference between gradients: 5.960464477539062e-07
Layer 10 	max difference between gradients: 3.5762786865234375e-07
Layer 11 	max difference between gradients: 0.0
Layer 12 	max difference between gradients: 0.0
Layer 13 	max difference between gradients: 0.0
Layer 14 	max difference between gradients: 0.0
Layer 15 	max difference between gradients: 0.0
Layer 16 	max difference between gradie

As you can see the difference of the gradients of the conv2d layers between the methods is (almost) numerically equivalent. The small differences are because of loss of significance with the floating points calculations. 

---

# Things to try:

* Use doubles instead of floats to reduce the difference (use model.double() and image_var.double())
* Make the image bigger than would fit on a GPU 
    - e.g. 8194x8194, make sure to add 3 more blocks in the model (see comments)
* If you want you can compare the reconstructed input gradients of each layer: 
    - pass fill_gradient=True in backward() function
    - compare full_gradients with self.model.gradients after the full model backward pass.
* For testing purposes the number of filters is small in this notebook, try increasing them